In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# **Overview**
This is a prediction of the sea ice extent (SIE) in 2030 from the northern hemisphere using a linear regression for the Daily Sea Ice Extent Dataset. Feel free to make suggestions or ask questions.

Consider upvoting this notebook, if you found it useful. 👍



# **Data Loading**

In [3]:
df = pd.read_csv('/kaggle/input/daily-sea-ice-extent-data/seaice.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/daily-sea-ice-extent-data/seaice.csv'

# **EDA**

In [ ]:
# first 5 rows of dataframe
df.head()

In [ ]:
# last 5 rows of dataframe
df.tail()

In [ ]:
# We only take data from the northern hemisphere
df = df[df.iloc[:,6]=='north']

In [ ]:
df.info()

In [ ]:
df.iloc[:,4].value_counts()

In [ ]:
corr = df.corr().drop(df.columns[[4]], axis=1)
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

We learn:

* there are no missing values
* the dataset has 13177 rows
* the dataset begins in October 1978 and ends in May 2019
* the columns of interest are year, month, day and extent
* The month variable is the most correlated with the sea ice extent (SIE)

In [ ]:
# Save the list of the years
years_list = list(df['Year'].value_counts().index)
years_list = sorted(years_list)

In [ ]:
# Calcule monthly average of sea ice extent
def by_months(df):
    n_months = 3 + ((2019-1979) * 12) + 5
    df_monthly = pd.DataFrame(columns=['Year', 'Month', 'Extent'], index=range(n_months))
    i = 0
    for year in years_list:

        if year == 1978:
            months_list = range(10,13)
        elif year == 2019:
            months_list = range(1,6)
        else:
            months_list = range(1,13)

        for month in months_list:
            subset = df[df['Year']==year][df.iloc[:,1]==month]
            monthly_mean = np.mean(subset.iloc[:,3])
            df_monthly.iloc[i] = [year, month, monthly_mean]
            i += 1

    return df_monthly

df_clean = by_months(df)

In [ ]:
df_clean.head()

In [ ]:
# Sort the dataset per month and year
df_clean = df_clean.sort_values(['Month', 'Year'])
df_clean = df_clean.reset_index().drop(['index'], axis=1)
df_clean.head()

In [ ]:
df_clean.tail()

In [ ]:
mean_years = []
std_dict = {}
for m in range(1,13):
  #Calcule mu and std values
  mu = df_clean[df_clean.Month == m]['Extent'].mean()
  std = df_clean[df_clean.Month == m]['Extent'].std()
  #Save mu and std values
  mean_years.append(mu)
  std_dict[m] = std

In [ ]:
df_mean_years = pd.DataFrame(mean_years)
df_mean_years['Month'] = range(1, 13)
df_mean_years.columns = ['Extent', 'Month']
plt.figure(figsize=(8, 8))
sns.scatterplot(x='Month', y='Extent', data=df_mean_years,  color='blue', sizes=20).set(title='Mean of SIE per month')
plt.show()

March is the month of the year with the most sea ice extent, and September the least. We use the data from the September months.

As a target variable for the linear regression, we calculate the SIE anomalies indicator, which is the difference between the extent for the month in question and the mean for that month based on the January 1981 to December 2010 data. This will then be converted to a percent difference by dividing it by the 1981-2010 average for that month and multiplying the result by 100.

In [ ]:
#Compute Sea Ice Extent anomalies
def anomaliesSIE(df):
  #Calcule mean by months from 1981-2010
  df_30y = df.query('1981 <= Year <= 2010')
  df_30y = df_30y.reset_index().drop(['index'], axis=1)
  mean_30y = []
  for m in range(1,13):
    mu = df[df.Month == m]['Extent'].mean()
    mean_30y.append(mu)

  #Compute SIE anomalies
  anomalies_list = []
  for s in range(0, len(df)):
    mean_30y_index = df.loc[s, 'Month']-1
    #Calcule anomalies
    anomalies_SIE = df.loc[s, 'Extent'] - mean_30y[mean_30y_index]
    anomalies_SIE = anomalies_SIE/mean_30y[mean_30y_index]*100
    #Save anomalies
    anomalies_list.append(anomalies_SIE)
  df['AnomaliesSIE'] = anomalies_list
  return df, mean_30y

df_SIE, mean_30y = anomaliesSIE(df_clean)
df_SIE.head()

In [ ]:
df_SIE['Year'] = pd.to_numeric(df_SIE['Year'])
#Total
sns.lmplot(x='Year', y='AnomaliesSIE', data=df_SIE, height=5.2, aspect=2).set(title='Total')
plt.show()
#January
sns.lmplot(x='Year', y='AnomaliesSIE', data=df_SIE[df_SIE.Month == 3], height=5.2, aspect=2).set(title='Mars')
plt.show()
#June
sns.lmplot(x='Year', y='AnomaliesSIE', data=df_SIE[df_SIE.Month == 9], height=5.2, aspect=2).set(title='September')
plt.show()

# Linear Regression Modelling

In [ ]:
y = np.asarray(df_SIE.AnomaliesSIE, dtype=float)  # response variable
x = np.asarray(df_SIE[['Year', 'Month']], dtype=float) # predictor
X = sm.add_constant(x)

In [ ]:
est = sm.OLS(y, X)

In [ ]:
est = est.fit()

In [ ]:
print(est.params)
print('MSE: ', est.mse_total)
print('R_squared: ', est.rsquared)

In [ ]:
#The trend expressed in percent per decade
est.params[1]*10

# Random forest


In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(x, y)
y_pred_rf = rf_model.predict(x)

In [ ]:
gbm_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm_model.fit(X, y)
y_pred_gbm = gbm_model.predict(X)

In [ ]:
with pm.Model() as model:
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta = pm.Normal('beta', mu=0, sigma=10, shape=(2,))
    sigma = pm.HalfNormal('sigma', sigma=1)
    mu = alpha + pm.math.dot(X, beta)
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=y)
    trace = pm.sample(500, return_inferencedata=False)
    y_pred_bayes = pm.sample_posterior_predictive(trace, samples=500)['Y_obs'].mean(axis=0)

In [ ]:
# Comparing the MSE of each model
mse_sm = results_sm.mse_total
mse_rf = mean_squared_error(y, y_pred_rf)
mse_gbm = mean_squared_error(y, y_pred_gbm)
mse_bayes = np.mean((y - y_pred_bayes) ** 2)

print('MSE for Statsmodels OLS:', mse_sm)
print('MSE for Random Forest:', mse_rf)
print('MSE for GBM:', mse_gbm)
print('MSE for Bayesian LR:', mse_bayes)

# Prediction

In [ ]:
est.predict()

In [ ]:
#Predict September 2025 anomaly
X_sep2030 = np.array([1, 2030, 9])
AnomaliesSIE_sep2030 = float(est.predict(X_sep2030))
AnomaliesSIE_sep2030

In [ ]:
#Calcule sea ice extent in September 2025
extent_sep2030 = (AnomaliesSIE_sep2030*mean_30y[8]/100)+mean_30y[8]
extent_sep2030 = round(extent_sep2030, 3)
extent_sep2030

In [ ]:
extent_sep1979 = df_clean[df_clean.Year == 1979].reset_index().loc[8, 'Extent']
print('sep_1979:', extent_sep1979, 'sep_2025:', extent_sep2030)

In [ ]:
#Calcule sea ice extent loss
loss_SIE = round(extent_sep1979-extent_sep2030,3)
loss_SIE

The Artic sea ice extent will be at its minimum of 5.008 millions km² million in 2030, which means the loss in 50 years will be 2.043 km² million, larger than the size of Mexico.